In [7]:
import sqlite3 as db
import datetime
from IPython.display import display, Javascript, Image
from ipywidgets import widgets
import time
from pop import Thermopile
import matplotlib
import matplotlib.font_manager as fm
import threading
%matplotlib inline
import pandas as pd             # 데이터를 저장하고 처리하는 패키지
import matplotlib as mpl        # 그래프를 그리는 패키지
import matplotlib.pyplot as plt # 그래프를 그리는 패키지
import multiprocessing as mp
import cv2
import numpy as np
from pop import Util
from pop import CO2

Util.enable_imshow()

lab = []
c = CO2(0)

In [8]:
#conn = db.connect("thermopile.db")
with db.connect("/home/soda/Project/python/notebook/CO2.db") as conn:
    conn.execute("""CREATE TABLE PersonTem
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    Temp REAL NOT NULL,
    date TEXT NOT NULL)""")

OperationalError: table PersonTem already exists

In [13]:
def database1(): # 데이터베이스에 센서 값 저장
    global d
    global lab
    with db.connect("/home/soda/Project/python/notebook/CO2.db") as conn:
        c=conn.cursor()
        data = c.calcPPM()
        now = datetime.datetime.now()
        nowDatetime = now.strftime('%m-%d %H:%M')
        lab[-1].value="CO2: {} [C], date : {}".format(data,nowDatetime)
        c.execute("INSERT INTO PersonTem(Temp,date) VALUES ('{}','{}') ".format(data,nowDatetime))
        conn.commit()

In [14]:
def dataconvert1(): # 데이터베이스에 있는 파일을 엑셀 파일로 변환
    with db.connect("/home/soda/Project/python/notebook/CO2.db") as conn:
        conn.row_factory = db.Row

        c = conn.cursor()
        tableAll=c.execute("SELECT * FROM PersonTem")

        row = c.fetchone()
        writeRow = ",".join([str(i) for i in row.keys()]) + '\n'

        with open("/home/soda/Project/python/notebook/data.csv","wb") as file:
            file.write(writeRow.encode())
            for data in tableAll:
                writeStr = [str(i).replace(',', '') for i in data]
                writeRow = ",".join(writeStr) + '\n'
                file.write(writeRow.encode())

In [15]:
def chartPrint():
    plt.clf()
    df = pd.read_csv('/home/soda/Project/python/notebook/data.csv', index_col='date',encoding='euc-kr')
    data_df = df.drop(columns='id')
    fm.get_fontconfig_fonts()
    font_location = '/usr/share/fonts/nanum/nanumGothicCoding.ttf'
    fprop = fm.FontProperties(fname=font_location)
    ax = data_df.plot(kind='line', title='CO2',figsize=(12,4),legend=True,fontsize=12)
    ax.set_xlabel('2021date', fontsize=12) # x축 정보 표시
    ax.set_ylabel('C02',fontsize=12) # y축 정보 표시
    ax.legend(['date/time','CO2'])
    ax.grid(True,axis='y')
    plt.xticks(rotation=45)
    plt.savefig('/home/soda/Project/python/notebook/line.png')
#     plt.show()

In [16]:
lab.append(widgets.Label(value="CO2: 0 [C]"))
display(lab[-1])
# dpi = matplotlib.rcParams['figure.dpi']
while True :
    database1()
    dataconvert1()
    chartPrint()
    image = cv2.imread("/home/soda/Project/python/notebook/line.png", cv2.IMREAD_COLOR)
#     cv2.imshow("image",image)
    cv2.imshow("image",image,width=780,height=900)
#     height, width, depth = image.shape
#     print("w %d,h %d,c %d"%(width, height, depth))
    time.sleep(2)

Label(value='CO2: 0 [C]')

AttributeError: 'sqlite3.Cursor' object has no attribute 'read'